In [2]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import ast
import re

In [2]:
standard = ['True', 'Mostly True ','Half True ', 'Mostly False', 'False', 'Pants on Fire']
score = np.array([10, 8, 6, 4, 2, 0])
poli_df = pd.read_csv('politifact_data.csv')
def transform_percentages(col):
    return sum(np.array(ast.literal_eval(col.replace('%', '').replace('\'', '').replace(' ', ',')))/100 * score)
poli_df['credibility_score'] = poli_df['percentages'].apply(transform_percentages)
poli_df['content'] = poli_df['content'].str.replace("“", "").str.replace("”","").str.replace("\"", '')
poli_df = poli_df.rename(columns = {'speaker':'recorder', 'media':'source'})

def extract_date(text):
    date_pattern = re.compile(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b')
    match = date_pattern.search(text) 
    if match:
        date = match.group()
        return date
poli_df['publish_time'] = poli_df['when/where'].apply(extract_date)
poli_df['content'].head()[0]
clean_df = poli_df[['source', 'credibility_score']].drop_duplicates().reset_index().drop(columns = 'index')
clean_df.head()
import requests
def search_wikipedia(query, num_results=15):
    endpoint_url = "https://en.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'srsearch': query,
        'srlimit': num_results
    }
    response = requests.get(endpoint_url, params=params)
    data = response.json()
    output = []
    search_results = data['query']['search']
    for result in search_results:
        title = result['title']
        output.append(title)
    return " ".join(output)
def scrape_titles(col):
    return search_wikipedia(col, num_results=20)
clean_df['source_history'] = clean_df['source'].apply(scrape_titles)
# clean_df.to_csv('credibility.csv')

In [3]:
clean_df = pd.read_csv('credibility.csv', index_col=None).drop(columns = 'Unnamed: 0')

In [4]:
clean_df.head()

,source,credibility_score,source_history
0,Instagram posts,1.74,Instagram List of most-followed Instagram acco...
1,Scott Walker,5.14,Scott Walker Scott Walker (politician) Scott W...
2,Viral image,1.60,Viral phenomenon Imgur The dress Midjourney Da...
3,Facebook posts,1.86,Facebook Facebook 3D Posts Meta Platforms Hist...
4,New York Citizens Audit,2.00,Sovereign citizen movement KPMG BNY Mellon Arc...


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [6]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [7]:
clean_df = clean_df[clean_df['source_history'].isna() == False]

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    tokens = nltk.word_tokenize(text)    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
clean_df['preprocessed_source'] = clean_df['source_history'].apply(preprocess_text)

In [9]:
X = clean_df['preprocessed_source']
y = clean_df['credibility_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    RandomForestRegressor(),
#     make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    BayesianRidge(),
]
for model in models:
    if model.__class__.__name__ != 'BayesianRidge':
        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
    else:    
        model.fit(X_train_tfidf.toarray(), y_train)
        y_pred = model.predict(X_test_tfidf.toarray())
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for {model.__class__.__name__}: {mse}')

Mean Squared Error for LinearRegression: 14.271328247245256
Mean Squared Error for Ridge: 8.287456497384378
Mean Squared Error for Lasso: 8.317311604743733
Mean Squared Error for RandomForestRegressor: 7.911604973033622
Mean Squared Error for BayesianRidge: 7.856691318441045


In [10]:
X_test = clean_df['source_history'][[100]]

In [11]:
X_test_tfidf = vectorizer.transform(X_test)

In [12]:
y_pred = model.predict(X_test_tfidf)